<a href="https://colab.research.google.com/github/Parths131/customer-segmentation/blob/main/customer_segmentation_phaseA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving online_retail_II.xlsx to online_retail_II.xlsx


In [ ]:
df = pd.read_excel('online_retail_II.xlsx', sheet_name='Year 2010-2011')

In [ ]:
print(df.shape)

(541910, 8)


In [ ]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      541910 non-null  object        
 1   StockCode    541910 non-null  object        
 2   Description  540456 non-null  object        
 3   Quantity     541910 non-null  int64         
 4   InvoiceDate  541910 non-null  datetime64[ns]
 5   Price        541910 non-null  float64       
 6   Customer ID  406830 non-null  float64       
 7   Country      541910 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB
None


In [ ]:
print(df.describe())

            Quantity                    InvoiceDate          Price  \
count  541910.000000                         541910  541910.000000   
mean        9.552234  2011-07-04 13:35:22.342307584       4.611138   
min    -80995.000000            2010-12-01 08:26:00  -11062.060000   
25%         1.000000            2011-03-28 11:34:00       1.250000   
50%         3.000000            2011-07-19 17:17:00       2.080000   
75%        10.000000            2011-10-19 11:27:00       4.130000   
max     80995.000000            2011-12-09 12:50:00   38970.000000   
std       218.080957                            NaN      96.759765   

         Customer ID  
count  406830.000000  
mean    15287.684160  
min     12346.000000  
25%     13953.000000  
50%     15152.000000  
75%     16791.000000  
max     18287.000000  
std      1713.603074  


In [ ]:
df = df.dropna(subset=['Customer ID'])

In [ ]:
print("Remaining rows:", df.shape[0])

Remaining rows: 406830


In [ ]:
df = df[~df['Invoice'].astype(str).str.startswith('C')]

In [ ]:
print("Rows after removing canceled invoices:", df.shape[0])

Rows after removing canceled invoices: 397925


In [ ]:
df['TotalPrice'] = df['Quantity'] * df['Price']

/tmp/ipython-input-12-2791139145.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TotalPrice'] = df['Quantity'] * df['Price']


In [ ]:
print(df[['Quantity', 'Price', 'TotalPrice']].head())

   Quantity  Price  TotalPrice
0         6   2.55       15.30
1         6   3.39       20.34
2         8   2.75       22.00
3         6   3.39       20.34
4         6   3.39       20.34


In [ ]:
import datetime as dt

In [ ]:
today_date = dt.datetime(2011, 12, 11)

In [ ]:
rfm = df.groupby('Customer ID').agg({
    'InvoiceDate': lambda x: (today_date - x.max()).days,
    'Invoice': 'nunique',
    'TotalPrice': 'sum'
})

In [ ]:
rfm.head()

,InvoiceDate,Invoice,TotalPrice
Customer ID,,,
12346.0,326,1,77183.60
12347.0,3,7,4310.00
12348.0,76,4,1797.24
12349.0,19,1,1757.55
12350.0,311,1,334.40


In [ ]:
rfm.describe()

,InvoiceDate,Invoice,TotalPrice
count,4339.000000,4339.000000,4339.000000
mean,93.041484,4.271952,2053.797166
std,100.007757,7.705493,8988.247836
min,1.000000,1.000000,0.000000
25%,18.000000,1.000000,307.245000
50%,51.000000,2.000000,674.450000
75%,142.500000,5.000000,1661.640000
max,374.000000,210.000000,280206.020000


In [ ]:
#rfm.to_csv('rfm.csv')

In [ ]:
rfm = rfm[rfm['TotalPrice'] > 0]